#### Creating Dataframe where each row has all the weather information from the website : https://www.estesparkweather.net/archive_reports.php from 2009 to 2021

In [ ]:
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle

In [ ]:
keys=['Average temperature (°F)', 'Average humidity (%)',
 'Average dewpoint (°F)', 'Average barometer (in)',
 'Average windspeed (mph)', 'Average gustspeed (mph)',
 'Average direction (°deg)', 'Rainfall for month (in)',
 'Rainfall for year (in)', 'Maximum rain per minute',
 'Maximum temperature (°F)', 'Minimum temperature (°F)',
 'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed (mph)',
 'Maximum gust speed (mph)', 'Maximum heat index (°F)']

keys.append('date')
keys

In [ ]:
day_data=dict.fromkeys(keys)

In [ ]:
day_data

In [ ]:
df = pd.DataFrame(columns=keys)
df.set_index('date',inplace=True)

In [ ]:
for year in range(2009,2022):
    for month in range(1,13):
        if month<10:
            search_date=str(year)+str(0)+str(month)
        else:
            search_date=str(year)+str(month)
        URL = "http://www.estesparkweather.net/archive_reports.php?date="+search_date
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-copy")
        tables = results.find_all("table")
        year = results.find("h1").text.split()[-1]
        day_data=dict.fromkeys(keys)
        for table in tables:
            trs=table.find_all("tr",class_='table-top')
            td=trs[0].find_all("td")
            date=td[0].text.split()
            if date[1].isdigit():
                day_data['date']=date[0]+" "+date[1]+" "+year
                trs = table.find_all("tr",class_='column-light')
                for tr in trs:
                    tds= tr.find_all("td")
                    r=re.compile(tds[0].text)
                    key=list(filter(r.match, keys))[0]
                    day_data[key]=tds[1].text.split()[0]
                trs = table.find_all("tr",class_='column-dark')
                for tr in trs:
                    tds= tr.find_all("td")
                    r=re.compile(tds[0].text)
                    key=list(filter(r.match, keys))[0]
                    day_data[key]=tds[1].text.split()[0]
                #print(len(day_data.keys()))
                temp_df=pd.DataFrame(day_data,index=['date'])
                temp_df.set_index('date',inplace=True)
                df=df.append(temp_df)

In [ ]:
df.head()

In [ ]:
df.shape